In [4]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

همه = 0
سیاسی = 1
اخبار سلامت = 39
واژه خانه = 43
که بوذ و چه کرد = 44
قرآن = 45
اقتصادی = 4
اجتماعی = 5
ورزشی = 6
سلامت = 25
بین الملل = 3
سیاست خارجی = 2
فرهنگی هنری = 8
محیط زیست = 29
فناوری = 14
علمی = 7
حوادث = 9
سرگرمی = 15
عمومی = 21
کاربران = 22
کافه گفت و گو = 23
سفر و تفریح = 24

In [5]:
def extract_news(start_date = '1401/01/01',end_date = '1401/01/28',subject_id = 0):
    upperframe=[] 
    page_number = 1
    url = 'https://www.asriran.com/fa/archive?service_id=1&sec_id=-1&cat_id='+str(subject_id)+'&rpp=20&from_date='+start_date+'&to_date='+end_date+'&p='
    while True:
        url1 = url + str(page_number)
        page = requests.get(url1)
        soup = BeautifulSoup(page.text, "html.parser")
        articles=soup.find_all('article',attrs={'vizhe_cv col-xs-12 col-sm-6'})
        print(page_number)
        if len(articles)==0 :
            print('finished')
            break
        
        for article in articles:
            title = article.find("a",attrs={'class':"vizhe_title"}).text.strip()
            link = article.find("a").get('href').strip()
            link = 'https://www.asriran.com' + link
            summary = article.find("div",attrs={'class':"vizhe_lead"}).text.strip()
            date = article.find("span",attrs={'class':"tarikh_archive"}).text.strip()

            url2 = link
            page2 = requests.get(url2)
            soup2 = BeautifulSoup(page2.text, "html.parser")

            subject = soup2.find("div",attrs={'class':"news_path"}).find_all("a")[-1].text
            
            body = soup2.find("div",attrs={'class':"body"}).text.strip()[:-155]

            news_id = soup2.find("div",attrs={'class':"news_nav news_id_c"}).text.strip()
            news_id = int(re.findall(r'\d+', news_id)[0])

            short_link = soup2.find("div",attrs={'class':"short-link row"}).find("a").get('href').strip()
            short_link = 'https://www.asriran.com' + short_link

            
            frame = [news_id,date,subject,title,summary,body,short_link]
            upperframe.append(frame)

        page_number += 1

    data=pd.DataFrame(upperframe, columns=['News_ID','Date','Subject','Title','Summary','Body','Short_link'])

    return data

In [6]:
data = extract_news(start_date = '1401/01/01',end_date = '1401/01/28',subject_id = 0)
data.head()

1
2
3
4
5
6
7
8
9
10


In [ ]:
data.to_csv('data1.csv', encoding='utf-8', index=False)